In [13]:
import sys
sys.path.append("/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts")

from typing import Union
from dataclasses import dataclass

from copy import deepcopy
import json

from case_edits.ezcase import EzCase

from recipes.amber import input, PLAN_PATH
from geometry.wall_normal import WallNormal
from helpers.helpers import key_from_value
from methods.subsurfaces.pairs import DEFAULT_WINDOW, SubsurfacePair, DEFAULT_DOOR




In [2]:
with open(PLAN_PATH) as f:
    plan_data = json.load(f)

In [3]:
room_map = {}
for item in plan_data[0]:
    room_map[item["id"]] = item["label"]

room_map

{0: 'bedroom_1',
 1: 'kitchen',
 2: 'm_bath',
 3: 'm_bedroom',
 4: 'm_closet',
 5: 'bath',
 6: 'dining',
 7: 'transit_space',
 8: 'closet',
 9: 'bedroom_2',
 10: 'laundry'}

In [4]:
CONNECTIVITY_PATH = "/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/svg2plan/outputs/amber_a/graph.json"

In [5]:
with open(CONNECTIVITY_PATH) as f:
    data = json.load(f)

In [6]:
@dataclass
class InputObject:
    node: Union[str, int]
    direction: str
    object_type: str

In [7]:
objects: list[InputObject] = []
for node in data["nodes"]:
    for w in node["windows"]:
        objects.append(InputObject(node["id"],
                        w["direction"],
                         w["window_type"]
        ))

In [8]:
WallNormal["NORTH"]

<WallNormal.NORTH: 0.0>

In [9]:
window_pairs: list[SubsurfacePair] = []
for o in objects:
    node = key_from_value(room_map, o.node) 
    direction = WallNormal[str(o.direction)]
    if o.object_type == "A":
        object_type = DEFAULT_WINDOW

    window_pairs.append(SubsurfacePair(node, direction, object_type))

In [11]:
doors = data["links"]
doors

[{'source': 'bedroom_1', 'target': 'transit_space'},
 {'source': 'kitchen', 'target': 'dining'},
 {'source': 'm_bath', 'target': 'm_bedroom'},
 {'source': 'm_bedroom', 'target': 'm_closet'},
 {'source': 'm_bedroom', 'target': 'dining'},
 {'source': 'bath', 'target': 'transit_space'},
 {'source': 'dining', 'target': 'transit_space'},
 {'source': 'dining', 'target': 'closet'},
 {'source': 'transit_space', 'target': 'bedroom_2'},
 {'source': 'transit_space', 'target': 'laundry'}]

In [14]:
door_pairs: list[SubsurfacePair] = []
for item in doors:
    door_pairs.append(SubsurfacePair(
        key_from_value(room_map, item["source"]),
        key_from_value(room_map, item["target"]),
        DEFAULT_DOOR
    )
    )

In [15]:
door_pairs

[SubsurfacePair(pair=(0, 7), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADING=False)),
 SubsurfacePair(pair=(1, 6), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADING=False)),
 SubsurfacePair(pair=(2, 3), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADING=False)),
 SubsurfacePair(pair=(3, 4), attrs = SubsurfaceAttributes(object_type=<SubsurfaceObjects.DOOR: 0>, construction=None, dimensions=Dimensions(width=0.3, height=0.9), location_in_wall=<NinePointsLocator.bottom_middle: 7>, FRACTIONAL=True, SHADI

In [7]:
ez = EzCase(input, RUN_CASE=False)
ez

tests/test21_amber
Added 5 walls 
Added 4 walls 
Added 4 walls 
Added 6 walls 
Added 4 walls 
Added 5 walls 
Added 7 walls 
Added 6 walls 
Added 4 walls 
Added 5 walls 
Added 5 walls 
out.idf does not exist
idf has changed - running case

/Applications/EnergyPlus-22-2-0/energyplus --weather /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/weather_data/PALO_ALTO/CA_PALO-ALTO-AP_724937_23.EPW --output-directory /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/tests/test21_amber/results --idd /Applications/EnergyPlus-22-2-0/Energy+.idd /Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/cases/base/01example/Minimal_AP_6b2b8a.idf

EnergyPlus Starting
EnergyPlus, Version 22.2.0-c249759bad, YMD=2024.09.04 16:59
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation
Reporting Surfaces
Beginning Primary Simulation
Initializing New Environment Parameters
Warming up {1}
Warming up {2}
Warming up {3}
Warming up {4}
Warming up {5}
War

EnergyPlus Completed Successfully.
/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts/outputs/sql.py:39: UserWarning:

AFN Zone Ventilation Sensible Heat Loss Rate not in available_outputs

/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/geomeppy/_scripts/analysis/post_process.py:28: UserWarning:

Output variables are not appropriate for this case



{'case_name': 'tests/test21_amber',
 'subsurface_pairs': [],
 'geometry': GPLANRoomAccess(path='/Users/julietnwagwuume-ezeoke/_UILCode/gqe-phd/svg2plan/outputs/amber_a/gplan.json',
                             index=0),
 'starting_case': '',
 'project_name': ''}

In [5]:
ez.show_base_plot()